# EXTRACCIÓN DE VUELOS

## En este Notebook se va a desarrollar el proceso de extracción de todos los vuelos salientes del *Aeropuerto Adolfo Suárez Madrid-Barajas* en el periodo comprendido entre el 01-11-2017 y el 31-10-2023, es decir, 1 año completo.

## Para ello se va a utilizar la técnica de extracción de datos "webscrapping" sobre la web [flightera](https://www.flightera.net/en/airport/Madrid/LEMD/departure) que mantiene registros de vuelos desde al menos el año 2017.

### Las librerías que se van a emplear principalmente para el proceso son:
- **selenium** 
- **pandas** 
- **joblib**

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from datetime import datetime, timedelta
import pandas as pd
import re

In [2]:
import threading
import time
import numpy as np
import pylab as plt   
import seaborn as sns
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime, timedelta


In [3]:
from joblib import Parallel, delayed
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select   # seleccion de un dropdown
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
import time
import pandas as pd
import multiprocessing as mp
import warnings
warnings.filterwarnings('ignore')

from joblib._parallel_backends import LokyBackend
import asyncio

from tqdm.notebook import tqdm


# Se generan los links de cada una de las páginas que se van a escrapear.

El link incluye la fecha del día a escrapear y la hora 00:00 para empezar a recoger desde el primer vuelo del día.

In [5]:
# # Definir las fechas de inicio y fin
# start_date = datetime(2017, 11, 1)
# end_date = datetime(2022, 11, 1)

# # Función para generar enlaces
# def generate_links(start_date, end_date):
#     links = []
    
#     # Iterar sobre las fechas en el rango
#     current_date = start_date
#     while current_date <= end_date:
#         # Formatear la fecha en el formato deseado
#         formatted_date = current_date.strftime("%Y-%m-%d")

#         # Construir el enlace y agregarlo a la lista
#         link = f"https://www.flightera.net/en/airport/Madrid/LEMD/departure/{formatted_date}%2000_00"
#         links.append(link)

#         # Avanzar a la siguiente fecha
#         current_date += timedelta(days=1)

#     return links

# # Generar la lista de enlaces
# links = generate_links(start_date, end_date)


In [10]:
links = ['https://www.flightera.net/en/airport/Madrid/LEMD/departure/2017-11-01%2000_00',
 'https://www.flightera.net/en/airport/Madrid/LEMD/departure/2018-1-01%2000_00',
 'https://www.flightera.net/en/airport/Madrid/LEMD/departure/2019-1-01%2000_00',
 'https://www.flightera.net/en/airport/Madrid/LEMD/departure/2020-1-01%2000_00',
 'https://www.flightera.net/en/airport/Madrid/LEMD/departure/2021-1-01%2000_00',
 'https://www.flightera.net/en/airport/Madrid/LEMD/departure/2022-1-01%2000_00']

In [9]:
match = re.search(r'\d{4}', links[0])
match = match.group()
match

'2017'

## Para evitar problemas en el proceso de extracción se van a activar las opciones de **adblock** (previa descarga del driver correspondiente) y de las **cockies**.

In [ ]:
# Options
opciones=Options()
opciones.add_extension('drivers/adblock.crx')       # adblocker
opciones.add_argument('cookies=cookies')    # man

In [ ]:
flights =[]

# La siguiente función desarrolla todo el proceso de extracción de datos para cada uno de los links generados. El proceso es el siguiente.

    - En cada una de las páginas de la web hay 35 registros de vuelos.
    - La función entra en la página del link donde se encuentran los primeros 35 vuelos de ese año.
    - Genera una tabla vacía y acepta las cookies.
    - Extrae los vuelos de la primera página del año y los añade a la tabla.
    - Clicka sobre *Later Flights* y cierra un pop-up que aparece siempre tras clickar por primera vez.
    - A partir de aquí inicializa un bucle repitiendo el proceso anterior hasta que el año en cuestión deja de aparecer en el link.
    -Una vez concluido devuelve la lista con todos los registros para ese día.

In [ ]:
def extraer(url):
    table = []  # Inicializa table como una lista vacía
    #Columnas tras la extracción
    #columns = ["bad_date","1", "status","cod_flight","airliner", "destiny", "depart","delay_d","8","arrival","delay_a","duration","12"]  # Añade las columnas
    #Fecha de la URL
     
    match = re.search(r'\d{4}', url)
    match = match.group()
    try:
        #Opciones
        opciones=Options()
        opciones.add_extension('drivers/adblock.crx')       # adblocker
        opciones.add_argument('cookies=cookies')    # man
#         opciones.add_argument(f'--load-extension={drivers/adblock.crx}')
        #Inicializo el drive
        driver = webdriver.Chrome(options=opciones)
        driver.get(url)
        wait = WebDriverWait(driver, 20)
        time.sleep(35)
        
        #Aceptar cookies
        
        aceptar = driver.find_element(By.XPATH,'//*[@id="qc-cmp2-ui"]/div[2]/div/button[2]')
        aceptar.click()
        
        # Extraer la información la primera vez
        table += [flight.text.split('\n') for flight in driver.find_elements(By.XPATH, '//table//tbody//tr')[1:]]
        time.sleep(2)
        
        # Clickar en el botón Later Flights para abrir una nueva URL
        aceptar = wait.until(EC.presence_of_element_located((By.XPATH, '/html/body/div[3]/main/div[6]/div[2]/a')))
        #aceptar = driver.find_element(By.XPATH, '/html/body/div[3]/main/div[6]/div[2]/a')
        aceptar.click()
        time.sleep(2)
        
        # Clickar en boton popup 
        aceptar2 = driver.find_element(By.XPATH, '/html/body/div[9]/div/div/div/div/div/div[3]/span[1]/button')
        aceptar2.click()
        time.sleep(2)
               
        aceptar = driver.find_element(By.XPATH,'/html/body/div[2]/main/div[6]/div[2]/a')
        aceptar.click()
        time.sleep(2)
        # Bucle while hasta que la fecha de cada link desaparezca de la URL

        while match in driver.current_url:
            try:
                #Extraer datos
                table += [flight.text.split('\n') for flight in driver.find_elements(By.XPATH, '//table//tbody//tr')[1:]]
                # Clickar en el botón Later Flights para abrir una nueva URL
                aceptar = driver.find_element(By.XPATH,'/html/body/div[2]/main/div[6]/div[2]/a')
                aceptar.click()
        
            except:
                # Clickar en boton popup si aparece
                aceptar2 = driver.find_element(By.XPATH,'/html/body/div[9]/div/div/div/div/div/div[3]/span[1]/button')
                aceptar2.click()       

        return table  # Devuelve un DataFrame con los datos de ese día 
        
    except Exception as e:
        print(f"Error: {e}")
        print(f"Error en la URL: {url}")
        
        # En caso de error, simplemente se devolverá la lista vacía 'table'
        return table 
    finally:
        driver.quit()

# La siguiente celda paraleliza el proceso de scrapeo anterior para poder escrapear los 6 años a la vez 

In [ ]:
%%time
paralelo = Parallel(n_jobs=6, verbose=True)

flights = paralelo(delayed(extraer)(url) for url in tqdm(links))

# Se convierte la lista de listas anterior en un dataframe

In [12]:
flights = [elemento for sublist in flights for elemento in sublist]
len(flights)

In [ ]:
pd.set_option('display.max_rows', None)
columns =["bad_date","1", "status","cod_flight","airliner", "destiny", "depart","delay_d","8","arrival","delay_a","duration",'12']

In [ ]:
flights = pd.DataFrame(flights, columns=columns)



In [ ]:
flights.head()

## Guardamos los datos en crudo en un archivo *.csv*

In [ ]:
flights.to_csv("../data/flights/raw_flights3.csv", index=False)